a 3stage hybrid IDS that incorporates a signature-based IDS is proposed to detectknown  attacks on vehicular networks

Training models- tree-based supervised learners — decision tree and extreme gradient boosting — used as multi-class classifiers for known attack detection;




In [ ]:
#importing all required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,precision_recall_fscore_support
from sklearn.metrics import f1_score,roc_auc_score
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
import FCBF_module
from xgboost import plot_importance
from sklearn.feature_selection import mutual_info_classif
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#dataset
attack_data = pd.read_csv("https://github.com/NirmalaKTomar/CICIDS2017_Sample_dataset/blob/main/CICIDS2017_sample.csv?raw=true");
attack_data.head

<bound method NDFrame.head of        Flow Duration  Total Fwd Packets  ...  Idle Min   Label
0                  4                  2  ...         0  BENIGN
1             142377                 46  ...         0  BENIGN
2             118873                 23  ...         0  BENIGN
3             143577                 43  ...         0  BENIGN
4             143745                 49  ...         0  BENIGN
...              ...                ...  ...       ...     ...
56656            234                  2  ...         0  BENIGN
56657         133288                  2  ...         0  BENIGN
56658       11507694                  5  ...   6503640     DoS
56659       11507707                  8  ...   6503122     DoS
56660       11512204                  8  ...   6507197     DoS

[56661 rows x 78 columns]>

In [ ]:
#identify types of attack
Detected_Types=pd.DataFrame(attack_data.Label.value_counts())
Detected_Types



,Label
BENIGN,22731
DoS,19035
PortScan,7946
BruteForce,2767
WebAttack,2180
Bot,1966
Infiltration,36


Data Pre-processing

In [ ]:
attributes = attack_data.dtypes[attack_data.dtypes != 'object'].index
attack_data[attributes] = attack_data[attributes].apply(lambda x: (x - x.mean()) / (x.std()))
attack_data = attack_data.fillna(0)

In [ ]:
#data sampling
labelencoder = LabelEncoder()
attack_data.iloc[:, -1] = labelencoder.fit_transform(attack_data.iloc[:, -1])#attack_data.Label.value_counts()

In [ ]:
attack_data_minor_category = attack_data[(attack_data['Label']==6)|(attack_data['Label']==1)|(attack_data['Label']==4)]
attack_data_major_category = attack_data.drop(attack_data_minor_category.index)

In [ ]:
X = attack_data_major_category.drop(['Label'],axis=1) 
y = attack_data_major_category.iloc[:, -1].values.reshape(-1,1)
y=np.ravel(y)

In [ ]:
from sklearn.cluster import MiniBatchKMeans
kmeans = MiniBatchKMeans(n_clusters=1000, random_state=0).fit(X)

In [ ]:
klabel=kmeans.labels_
attack_data_major_category['klabel']=klabel

In [ ]:
attack_data_major_category['klabel'].value_counts()

608    1037
21      936
625     822
0       811
935     694
       ... 
435       1
578       1
109       1
449       1
708       1
Name: klabel, Length: 993, dtype: int64

In [ ]:
cols = list(attack_data_major_category)
cols.insert(78, cols.pop(cols.index('Label')))
attack_data_major_category= attack_data_major_category.loc[:, cols]

In [ ]:
def typicalSampling(group):
    name = group.name
    frac = 0.008
    return group.sample(frac=frac)

resultant_data = attack_data_major_category.groupby('klabel', group_keys=False).apply(typicalSampling)

In [ ]:
resultant_data['Label'].value_counts()

0    133
3    101
5     61
2     20
Name: Label, dtype: int64

In [ ]:
resultant_data

,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,klabel,Label
16330,-0.528467,-0.086001,-0.066771,-0.034212,-0.048288,-0.245670,-0.169382,-0.236405,-0.255104,-0.557556,-0.289423,-0.591663,-0.528223,0.0,0.0,-0.383672,-0.489378,-0.486916,-0.054946,-0.520668,-0.392488,-0.468399,-0.482885,-0.094671,-0.326264,-0.216404,-0.256196,-0.276215,-0.087937,-0.200224,0.0,-0.004201,0.0,-0.111779,-0.092709,-0.220606,0.078607,-0.339802,-0.567368,-0.619817,-0.568382,-0.431131,-0.250277,-0.200224,-0.011883,1.225473,-0.758865,-0.281049,-0.004201,-0.011883,0.585968,-0.620067,-0.236405,-0.591663,-0.111779,0.0,0.0,0.0,0.0,0.0,0.0,-0.086001,-0.034212,-0.066771,-0.048335,-0.524289,-0.257549,-0.058528,-0.434903,-0.109889,-0.081786,-0.128620,-0.094860,-0.458729,-0.137651,-0.466440,-0.442057,0,5
10682,-0.528467,-0.086001,-0.066771,-0.034212,-0.048288,-0.245670,-0.169382,-0.236405,-0.255104,-0.557556,-0.289423,-0.591663,-0.528223,0.0,0.0,-0.383671,-0.489378,-0.486916,-0.054945,-0.520668,-0.392488,-0.468399,-0.482885,-0.094671,-0.326264,-0.216404,-0.256196,-0.276215,-0.087937,-0.200224,0.0,-0.004201,0.0,-0.111779,-0.092709,-0.222481,0.066179,-0.339802,-0.567368,-0.619817,-0.568382,-0.431131,-0.250277,-0.200224,-0.011883,1.225473,-0.758865,-0.281049,-0.004201,-0.011883,0.585968,-0.620067,-0.236405,-0.591663,-0.111779,0.0,0.0,0.0,0.0,0.0,0.0,-0.086001,-0.034212,-0.066771,-0.048335,-0.524289,-0.257549,-0.058528,-0.434903,-0.109889,-0.081786,-0.128620,-0.094860,-0.458729,-0.137651,-0.466440,-0.442057,0,5
16280,-0.528468,-0.086001,-0.066771,-0.034212,-0.048288,-0.245670,-0.169382,-0.236405,-0.255104,-0.557556,-0.289423,-0.591663,-0.528223,0.0,0.0,-0.383675,-0.489378,-0.486916,-0.054950,-0.520668,-0.392488,-0.468399,-0.482885,-0.094671,-0.326264,-0.216404,-0.256196,-0.276215,-0.087937,-0.200224,0.0,-0.004201,0.0,-0.111779,-0.092709,-0.212448,0.132686,-0.339802,-0.567368,-0.619817,-0.568382,-0.431131,-0.250277,-0.200224,-0.011883,1.225473,-0.758865,-0.281049,-0.004201,-0.011883,0.585968,-0.620067,-0.236405,-0.591663,-0.111779,0.0,0.0,0.0,0.0,0.0,0.0,-0.086001,-0.034212,-0.066771,-0.048335,-0.524289,-0.257549,-0.058528,-0.434903,-0.109889,-0.081786,-0.128620,-0.094860,-0.458729,-0.137651,-0.466440,-0.442057,0,5
10603,-0.528468,-0.086001,-0.066771,-0.034212,-0.048288,-0.245670,-0.169382,-0.236405,-0.255104,-0.557556,-0.289423,-0.591663,-0.528223,0.0,0.0,-0.383675,-0.489378,-0.486916,-0.054950,-0.520668,-0.392488,-0.468399,-0.482885,-0.094671,-0.326264,-0.216404,-0.256196,-0.276215,-0.087937,-0.200224,0.0,-0.004201,0.0,-0.111779,-0.092709,-0.213132,0.128153,-0.339802,-0.567368,-0.619817,-0.568382,-0.431131,-0.250277,-0.200224,-0.011883,1.225473,-0.758865,-0.281049,-0.004201,-0.011883,0.585968,-0.620067,-0.236405,-0.591663,-0.111779,0.0,0.0,0.0,0.0,0.0

Data splitting

In [ ]:
X = attack_data.drop(['Label'],axis=1).values
y = attack_data.iloc[:, -1].values.reshape(-1,1)
y=np.ravel(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size = 0.8, test_size = 0.2, random_state = 0,stratify = y)

In [ ]:
#attribute selection process
dominants = mutual_info_classif(X_train, y_train)

In [ ]:
attribute_list = sorted(zip(map(lambda x: round(x, 4),dominants), attributes), reverse=True)
Sum = 0
selected_attributes = []
for i in range(0, len(attribute_list)):
    Sum = Sum + attribute_list[i][0]
    selected_attributes .append(attribute_list[i][1])

In [ ]:
# selecting the dominant attributes 
dominant_attribute_list = sorted(zip(map(lambda x: round(x, 4), dominants/Sum), attributes), reverse=True)
Sum2 = 0
selected_attributes = []
for i in range(0, len(dominant_attribute_list)):
    Sum2 = Sum2 + dominant_attribute_list[i][0]
    selected_attributes.append(dominant_attribute_list[i][1])
    if Sum2>=0.9:
        break

In [ ]:
selected_attributes_data = attack_data[selected_attributes ].values

In [ ]:
#attributes selection by Fast Correlation Based Filter (FCBF)
from FCBF_module import FCBF, FCBFK, FCBFiP, get_i
fcbf = FCBFK(k = 20)

In [ ]:
selected_attributes_list = fcbf.fit_transform(selected_attributes_data,y)

In [ ]:
selected_attributes_list.shape

(56661, 20)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(selected_attributes_list,y, train_size = 0.8, test_size = 0.2, random_state = 0,stratify = y)

In [ ]:
X_train.shape

(45328, 20)

In [ ]:
pd.Series(y_train).value_counts()

0    18184
3    15228
5     6357
2     2213
6     1744
1     1573
4       29
dtype: int64

In [ ]:
from imblearn.over_sampling import SMOTE
smote=SMOTE(n_jobs=-1,sampling_strategy='auto')

In [ ]:
X_train, y_train = smote.fit_sample(X_train, y_train)

In [ ]:
pd.Series(y_train).value_counts()

6    18184
5    18184
4    18184
3    18184
2    18184
1    18184
0    18184
dtype: int64

Descision Tree classifier

In [ ]:
des_tree_model = DecisionTreeClassifier(random_state = 0)
des_tree_model.fit(X_train,y_train) 
des_tree_score=des_tree_model.score(X_test,y_test)
y_predict=des_tree_model.predict(X_test)
y_true=y_test
print('Accuracy of Descision Tree : '+ str(des_tree_score))
precision,recall,fscore,none= precision_recall_fscore_support(y_true, y_predict, average='weighted') 
print('Precision = '+(str(precision)))
print('Recall = '+(str(recall)))
print('F1-score = '+(str(fscore)))
print(classification_report(y_true,y_predict))

Accuracy of Descision Tree : 0.9927644930733257
Precision = 0.9934159851599265
Recall = 0.9927644930733257
F1-score = 0.9930070318454302
              precision    recall  f1-score   support

           0       1.00      0.99      0.99      4547
           1       0.98      0.99      0.98       393
           2       1.00      1.00      1.00       554
           3       1.00      1.00      1.00      3807
           4       0.31      0.71      0.43         7
           5       0.99      1.00      1.00      1589
           6       0.95      0.99      0.97       436

    accuracy                           0.99     11333
   macro avg       0.89      0.95      0.91     11333
weighted avg       0.99      0.99      0.99     11333



In [ ]:
dt_train=des_tree_model.predict(X_train)
dt_test=des_tree_model.predict(X_test)

In [ ]:
#extreme gradient boostclassifier

In [ ]:
ex_grad_classifier = xgb.XGBClassifier(n_estimators = 10)
ex_grad_classifier.fit(X_train,y_train)
ex_grad_classifier_score=ex_grad_classifier.score(X_test,y_test)
y_predict=ex_grad_classifier.predict(X_test)
y_true=y_test
print('Accuracy of ex_grad_classifierBoost: '+ str(ex_grad_classifier_score))
precision,recall,fscore,none= precision_recall_fscore_support(y_true, y_predict, average='weighted') 
print('Precision = '+(str(precision)))
print('Recall = '+(str(recall)))
print('F1-score = '+(str(fscore)))
print(classification_report(y_true,y_predict))

Accuracy of ex_grad_classifierBoost: 0.8824671313862172
Precision = 0.9223603306506323
Recall = 0.8824671313862172
F1-score = 0.8923539742890073
              precision    recall  f1-score   support

           0       0.97      0.76      0.85      4547
           1       0.51      0.99      0.68       393
           2       0.63      1.00      0.77       554
           3       0.93      0.95      0.94      3807
           4       0.02      0.57      0.04         7
           5       0.99      0.99      0.99      1589
           6       0.83      0.89      0.86       436

    accuracy                           0.88     11333
   macro avg       0.70      0.88      0.73     11333
weighted avg       0.92      0.88      0.89     11333



In [ ]:
xg_train=ex_grad_classifier.predict(X_train)
xg_test=ex_grad_classifier.predict(X_test)


Apply stacking

In [ ]:
base_predictions_train = pd.DataFrame( {'DecisionTree': dt_train.ravel(),'ex_grad_classifier': xg_train.ravel(),})
base_predictions_train.head(5)

,DecisionTree,ex_grad_classifier
0,5,5
1,3,3
2,5,5
3,3,3
4,2,2


In [ ]:
#trainging 
dt_train=dt_train.reshape(-1, 1)
xg_train=xg_train.reshape(-1, 1)
#testing
dt_test=dt_test.reshape(-1, 1)
xg_test=xg_test.reshape(-1, 1)

In [ ]:
x_train = np.concatenate(( dt_train,xg_train), axis=1)
x_test = np.concatenate(( dt_test,xg_test), axis=1)

In [ ]:
stk = xgb.XGBClassifier().fit(x_train, y_train)
y_predict=stk.predict(x_test)
y_true=y_test
stk_score=accuracy_score(y_true,y_predict)
print('Accuracy of Stacking: '+ str(stk_score))
precision,recall,fscore,none= precision_recall_fscore_support(y_true, y_predict, average='weighted') 
print('Precision of Stacking: '+(str(precision)))
print('Recall of Stacking: '+(str(recall)))
print('F1-score of Stacking: '+(str(fscore)))
print(classification_report(y_true,y_predict))


Accuracy of Stacking: 0.9927644930733257
Precision of Stacking: 0.9934159851599265
Recall of Stacking: 0.9927644930733257
F1-score of Stacking: 0.9930070318454302
              precision    recall  f1-score   support

           0       1.00      0.99      0.99      4547
           1       0.98      0.99      0.98       393
           2       1.00      1.00      1.00       554
           3       1.00      1.00      1.00      3807
           4       0.31      0.71      0.43         7
           5       0.99      1.00      1.00      1589
           6       0.95      0.99      0.97       436

    accuracy                           0.99     11333
   macro avg       0.89      0.95      0.91     11333
weighted avg       0.99      0.99      0.99     11333

